The TensorFlow tutorial: Deep MNIST for Experts

What will be accomplished:
<ul>
<li>Create a softmax regression function that is a model for recognizing MNIST digits, based on looking at every pixel in the image</li>
<li>Use Tensorflow to train the model to recognize digits by having it "look" at thousands of examples</li>
<li>Check the model's accuracy with test data</li>
<li>Build, train, and test a multilayer convolutional neural network to improve the results</li>
</ul>

Load the MNIST data

In [1]:
#Load the MNIST Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Start a TensorFlow InteractiveSession. This allows graph and session elements to be coded together, instead of separately.

In [4]:
import tensorflow as tf
sess = tf.InteractiveSession()


Define some helper functions to create weights and biases for the CNN.

In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

Add functions to create CNN layers.

In [6]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

Building the computation graph to be run. 

In [8]:
#placeholders for the inputs
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

#Conv Layer 1
#the weights 
# [patch size, patch size, input channels, output channels]
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

#reshape x to fit a 4d tensor, with the second and third dimensions being image width and height
# final dimension the number of color chanels
x_image = tf.reshape(x, [-1, 28, 28, 1])

#convolving x_image with the weight tensor, add the bias, applly the ReLU function, then max pool
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#Conv Layer 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#Densely connected layer
#image size has been reduced to 7x7. We add a fully-connected layer with 1024 units
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#Dropout to reduce overfitting
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#Readout layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2



Training the model. We need to define a training step, then execute this using batches of our data. 

In [10]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(20000):
  batch = mnist.train.next_batch(10)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 100, training accuracy 0.5
step 200, training accuracy 0.8
step 300, training accuracy 0.9
step 400, training accuracy 0.9
step 500, training accuracy 1
step 600, training accuracy 0.9
step 700, training accuracy 0.9
step 800, training accuracy 1
step 900, training accuracy 0.9
step 1000, training accuracy 0.9
step 1100, training accuracy 0.9
step 1200, training accuracy 1
step 1300, training accuracy 1
step 1400, training accuracy 1
step 1500, training accuracy 1
step 1600, training accuracy 1
step 1700, training accuracy 0.9
step 1800, training accuracy 0.9
step 1900, training accuracy 1
step 2000, training accuracy 1
step 2100, training accuracy 0.9
step 2200, training accuracy 1
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 0.9
step 2600, training accuracy 0.8
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 0.8
step 3000, training accuracy 1
step 3100, training

Evaluating the model.